In [16]:
import pandas as pd

leases_df = pd.read_csv("../data/Leases.csv")


# Removal Step

In [ ]:
# Drop na rows for company name
# leases_df = pd.DataFrame(leases_df, columns=["company_name"]).dropna()

# Omit rows with na company_name
leases_df = leases_df[leases_df["company_name"].notna()]

# Remove columns with all na entries
remove_features = [
    "direct_available_space",
    "direct_availability_proportion",
    "direct_internal_class_rent",
    "direct_overall_rent",
    "sublet_available_space",
    "sublet_availability_proportion",
    "sublet_internal_class_rent",
    "sublet_overall_rent"
]
for feature in remove_features:
    leases_df = leases_df[leases_df[feature].notna()]

23595

# Imputation Step
Impute numerical values in dataset with median
https://scikit-learn.org/stable/modules/impute.html
https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [ ]:
# Categorical/String
# company_name	internal_industry internal_market_cluster

# Numerical
# RBA	available_space	availability_proportion	internal_class_rent	overall_rent


# Export Cleaned Data

In [21]:
leases_df.to_csv("../data/cleaned_leases.csv", index=False)